# More on classes

We begin with review and design considerations.

In [1]:
class Widget: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [2]:
Widget('10 years', '10 pounds')

Widget('10 years', '10 pounds')

In [3]:
Widget(10.0, 1.0) == Widget(10, 1)

True

In [4]:
Widget(10.0, 1.0) == Widget(10, 2)

False

In [5]:
class Employee: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [6]:
Employee(23.0, 200.0) == Employee(23, 200)

True

In [7]:
Employee(23.0, 200.0) == Employee(20, 200)

False

In [8]:
w = Widget(70, 180)

In [9]:
w == 42

False

In [10]:
e = Employee(70, 180)

In [11]:
w == e

False

In [12]:
class ExplodingWidget(Widget):
    def explode(self):
        print(f'{self!r} has exploded!')

In [13]:
ew = ExplodingWidget(70, 180)

In [14]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [15]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [16]:
w == ew

True

In [17]:
w.__dict__ == e.__dict__

True

## Weird derived classes (semi-review)

In [18]:
class CallableStr(str): 
    def __call__(self): 
        return self

In [19]:
f = CallableStr(312)

In [20]:
f

'312'

In [21]:
f()

'312'

In [22]:
f is f()

True

In [23]:
from algoviz.decorators import joining

In [24]:
joining(f)

<function __main__.joining.<locals>.decorator.<locals>.wrapper()>

In [25]:
joining('s')

<function algoviz.decorators.joining.<locals>.decorator(func)>

## Effect of defining special methods (review)

Defining special dunder methods ("magic" methods) as attributes of a class (which is most often done with a `def` statement in the class body) customizes the behavior of *instances* of the class. They should not be defined on the instances themselves (which does not work, and constitutes using a dunder name in a manner other than explicitly documented).

In [26]:
class HasRepr: 
    def __repr__(self): 
        return f'{type(self).__name__}()'

In [27]:
hr = HasRepr()

In [28]:
hr.__repr__()

'HasRepr()'

In [29]:
hr

HasRepr()

In [30]:
repr(hr)

'HasRepr()'

In [31]:
class NoRepr: 
    pass

In [32]:
nr = NoRepr()
nr.__repr__ = lambda self: f'{type(self).__name__}()'

In [33]:
nr.__repr__(nr)

'NoRepr()'

In [34]:
NoRepr.__repr__ = lambda self: f'{type(self).__name__}()'

In [35]:
nr

NoRepr()

In [36]:
nr.__repr__()

TypeError: <lambda>() missing 1 required positional argument: 'self'

In [37]:
nr2 = NoRepr()

In [38]:
nr2.__repr__()

'NoRepr()'

In [39]:
class ReallyNoRepr: 
    pass

In [40]:
rnr = ReallyNoRepr()
rnr.__repr__ = lambda: f'{type(rnr).__name__}()'

In [41]:
rnr.__repr__()

'ReallyNoRepr()'

In [42]:
rnr

## Review of `super`

In [43]:
del CallableStr  # Just to make clear that we're not reusing the old one.

In [44]:
class CallableStr(str): 
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'
    
    def __call__(self): 
        return self

In [45]:
f = CallableStr(312)

In [46]:
f

CallableStr('312')

In [47]:
CallableStr('312')

CallableStr('312')

In [48]:
CallableStr("a'b")

CallableStr("a'b")

In [49]:
CallableStr('a\'b')

CallableStr("a'b")

In [50]:
repr(f)

"CallableStr('312')"

In [51]:
f.__repr__()

"CallableStr('312')"

In [52]:
CallableStr.__repr__(f)

"CallableStr('312')"

### Zero-argument `super` only works in a function defined in a class.

In [53]:
kent = super()

RuntimeError: super(): no arguments

### The technique is not limited to `str`.

In [54]:
class CallableInt(int): 
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'
    
    def __call__(self): 
        return self

In [55]:
g = CallableInt('312')

In [56]:
g

CallableInt(312)

In [57]:
CallableInt(312)

CallableInt(312)

## Conferring that style of `repr` via mixin class

In [58]:
class _MixinRepr:
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'

In [59]:
class CallableStrA(_MixinRepr, str): 
    def __call__(self): 
        return self

In [60]:
a = CallableStrA('a')

In [61]:
a

CallableStrA('a')

In [62]:
class CallableIntA(_MixinRepr, int): 
    def __call__(self): 
        return self

In [63]:
one = CallableIntA('1')

In [64]:
one

CallableIntA(1)

In [65]:
class BrokenCallableStrA(str, _MixinRepr):  # Lists bases in wrong order.
    def __call__(self): 
        return self

In [66]:
BrokenCallableStrA('a')

'a'

## Important sidebar: Method resolution order

In [67]:
CallableStrA.__mro__

(__main__.CallableStrA, __main__._MixinRepr, str, object)

In [68]:
BrokenCallableStrA.__mro__

(__main__.BrokenCallableStrA, str, __main__._MixinRepr, object)

## Sidebar: Ways to access MRO and base class information

In [69]:
CallableIntA.__mro__  # The way you usually should access the MRO.

(__main__.CallableIntA, __main__._MixinRepr, int, object)

In [70]:
CallableIntA.mro()  # This is used during class creation to populate __mro__.

[__main__.CallableIntA, __main__._MixinRepr, int, object]

In [71]:
type.__mro__

(type, object)

In [72]:
type.mro()

TypeError: unbound method type.mro() needs an argument

In [73]:
type(type)

type

In [74]:
type.mro(type)

[type, object]

In [75]:
CallableIntA.__bases__

(__main__._MixinRepr, int)

In [76]:
CallableIntA.__base__  # Not very useful (unless single inheritance is known).

int

In [77]:
_MixinRepr.__subclasses__()

[__main__.CallableStrA, __main__.CallableIntA, __main__.BrokenCallableStrA]

## What `super` does

In [78]:
help(super)

Help on class super in module builtins:

class super(object)
 |  super() -> same as super(__class__, <first argument>)
 |  super(type) -> unbound super object
 |  super(type, obj) -> bound super object; requires isinstance(obj, type)
 |  super(type, type2) -> bound super object; requires issubclass(type2, type)
 |  Typical use to call a cooperative superclass method:
 |  class C(B):
 |      def meth(self, arg):
 |          super().meth(arg)
 |  This works for class methods too:
 |  class C(B):
 |      @classmethod
 |      def cmeth(cls, arg):
 |          super().cmeth(arg)
 |  
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner=None, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ---------------------

In [79]:
class _MixinReprAlt:
    def __repr__(self): 
        return f'{type(self).__name__}({super(_MixinReprAlt, self).__repr__()})'

In [80]:
class CallableStrB(_MixinReprAlt, str): 
    def __call__(self): 
        return self

In [81]:
class CallableIntB(_MixinReprAlt, int): 
    def __call__(self): 
        return self

In [82]:
CallableStrB(42)

CallableStrB('42')

In [83]:
CallableIntB('42')

CallableIntB(42)

In [84]:
CallableIntB.__mro__

(__main__.CallableIntB, __main__._MixinReprAlt, int, object)

In [85]:
_MixinReprAlt.__mro__

(__main__._MixinReprAlt, object)

## `super` in a class method

In [86]:
class HasClassMethod: 
    a = 3
    
    @classmethod
    def show_class_dict(cls):
        print(cls.__dict__)

In [87]:
hcm = HasClassMethod()

In [88]:
hcm.show_class_dict()

{'__module__': '__main__', 'a': 3, 'show_class_dict': <classmethod(<function HasClassMethod.show_class_dict at 0x0000018D8C179360>)>, '__dict__': <attribute '__dict__' of 'HasClassMethod' objects>, '__weakref__': <attribute '__weakref__' of 'HasClassMethod' objects>, '__doc__': None}


In [89]:
class UsesSuper(HasClassMethod): 
    a = 5
    
    @classmethod
    def use_super(cls): 
        print(super().a)

In [90]:
us = UsesSuper()

In [91]:
us.a

5

In [92]:
us.use_super()

3


In [93]:
class UsesSuperAlt(HasClassMethod): 
    a = 5
    
    @classmethod
    def use_super(cls): 
        print(super(UsesSuperAlt, cls).a)

In [94]:
usa = UsesSuperAlt()

In [95]:
usa.a

5

In [96]:
usa.use_super()

3


## `super` outside a class

In [97]:
def _add_repr_bad(cls): 
    """Decorator to add repr to a class based on superclass."""
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'
    
    cls.__repr__ = __repr__
    return cls

In [98]:
@_add_repr_bad
class BadCallableStrC(str): 
    def __call__(self): 
        return self

In [99]:
@_add_repr_bad
class BadCallableIntC(int): 
    def __call__(self): 
        return self

In [100]:
BadCallableStrC(21)

RuntimeError: super(): __class__ cell not found

In [101]:
def _add_repr(cls): 
    """Decorator to add repr to a class based on superclass."""
    def __repr__(self): 
        return f'{type(self).__name__}({super(cls, self).__repr__()})'
    
    cls.__repr__ = __repr__
    return cls

In [102]:
@_add_repr
class CallableStrC(str): 
    def __call__(self): 
        return self

In [103]:
@_add_repr
class CallableIntC(int): 
    def __call__(self): 
        return self

In [104]:
CallableStrC(365)

CallableStrC('365')

In [105]:
CallableIntC('45567')

CallableIntC(45567)

## `super` is not *usually* useful in a static method

In [106]:
class HasA: 
    a = 3

In [107]:
class HasSM(HasA): 
    @staticmethod
    def show_a(): 
        print(super().a)

In [108]:
hsm = HasSM()

In [109]:
hsm.show_a()

RuntimeError: super(): no arguments

In [110]:
class HasSM2(HasA): 
    @staticmethod
    def show_a_scaled(n): 
        print(super().a * n)

In [111]:
hsm2 = HasSM2()

In [112]:
hsm2.show_a_scaled(3)

TypeError: super(type, obj): obj must be an instance or subtype of type

## Customizing construction: `__new__`

In [113]:
class CustomNew: 
    def __new__(cls): 
        print("Constructing...")
        return super().__new__(cls)

In [114]:
CustomNew()

Constructing...


In [115]:
class Derived(CustomNew): 
    pass

In [116]:
Derived()

Constructing...


In [117]:
class WeirdBase: 
    def __new__(cls): 
        print("Constructing...")
        return super().__new__(WeirdBase)

In [118]:
class WeirdDerived(WeirdBase): 
    pass

In [119]:
WeirdDerived()

Constructing...


## `__new__` is a static method

`__new__` uses the object creation logic of the class it is called *on* to create an instance of the class passed as its first parameter, which should be called `cls`. At runtime, these classes may be the same or different. In particular, if `__new__` is not implemented anywhere lower in your class's inheritance hierarchy, then the implementation inherited from `object` is used to create an instance of an object of your type.

**`__new__` is a static method.** It is *implicitly* a static method: you do not need to, and never should, decorate it `@staticmethod`. Due to its `cls` parameter, people often wrongly think `__new__` is a class method. But a class method would automatically bind the class it is called on&mdash;or the class of the instance it is called on&mdash;which must not happen here. The point is to be able to construct an direct instance of the class we call `__new__` on, or *any* of its direct or indirect subclasses. So the class we call `__new__` on, and the class we pass as the first parameter (the `cls` parameter), must be separately specified.

`__new__` is the only common case of a *static method* where it makes sense to write `super()` in it. Recall that the zero-argument form of `super` often makes sense in instance methods and class methods, but rarely in static methods.

Calling `C.__new__(cls)`, where `cls` is not `C` or a direct or indirect subclass of `C`, raises `TypeError`.

## `__new__` and `__init__`: Explanation

`__new__` actually creates the object (or delegates to another function, almost always a base-class implementation of `__new__`, to do so). `__init__` initializes the object. The object, upon being returned by `__new__`, may or may not have all state set as needed. If not, `__init__` should take care of that.

You can set whatever state you want on an object in `__new__`, and it tends to be simpler to implement only `__new__` than to implement *both* `__new__` and `__init__`. But you can have both, and this is occasionally useful.

Furthermore, and most importantly, if all you need to do is to initialize an object by setting state on it&mdash;rather than creating the object in a specialized way&mdash;you should usually implement just `__init__` and not `__new__`. We implement `__new__` *far* less often than `__init__` when writing classes in Python.

## `__new__` and `__init__`: Simple example

In [120]:
class HasNewAndInit: 
    def __new__(cls): 
        print("Constructing...")
        return super().__new__(cls)
    
    def __init__(self):
        print("Constructed.")

In [121]:
HasNewAndInit()

Constructing...
Constructed.


## `__new__` with arguments (in addition to `cls`)

In [122]:
class HasNewInitAndArg: 
    def __new__(cls, arg): 
        print(f"Constructing with arg {arg}.")
        return super().__new__(cls)
    
    def __init__(self, arg):
        print(f"Constructed with arg {arg}.")

In [123]:
HasNewInitAndArg(3)

Constructing with arg 3.
Constructed with arg 3.


In [124]:
class DerivedTwoArgs(HasNewInitAndArg): 
    def __init__(self, arg1, arg2): 
        super().__init__(arg1 * arg2)

In [125]:
DerivedTwoArgs(3, 4)

TypeError: HasNewInitAndArg.__new__() takes 2 positional arguments but 3 were given

In [126]:
class FlexibleNewInitAndArg: 
    def __new__(cls, arg, *args, **kwargs):
        print(f"Constructing with arg {arg}.")
        
        # Note: We should pass *args, **kwargs if the base __new__ can use them.
        return super().__new__(cls)
    
    def __init__(self, arg):
        print(f"Constructed with arg {arg}.")

In [127]:
class DerivedFromFlex(FlexibleNewInitAndArg): 
    def __init__(self, arg1, arg2): 
        super().__init__(arg1 * arg2)

In [128]:
DerivedFromFlex(3, 4)

Constructing with arg 3.
Constructed with arg 12.


## When `__init__` is/isn't called after `__new__`

When a class is called, the logic of `type.__call__` (which, at least in CPython, is actually implemented in C) runs. That logic is **approximately** as follows:

```python
class type:
    ...
    
    # This is an instance method of type, so self is a class.
    def __call__(self, *args, **kwargs):
        instance = self.__new__(self, *args, **kwargs)
        if isinstance(instance, self):
            instance.__init__(*args, **kwargs)
        return instance
    
    ...
```

That is, when you call a class:

1. **`__new__` is called on the class.** If the class does not override `__new__`, an inherited implementation is selected (often `object.__new__`).

   Any arguments you passed when calling the class are forwarded to `__new__` in this call.

2. **If the returned object is really a (direct or indirect) instance of the class you called**, `__init__` is called *on* that object.

   Any arguments you passed when calling the class are forwarded to `__init__` in this call.

3. The object returned by `__new__`, whether or not `__init__` was subsequently called on it, is returned.

Of course, if an exception is raised at any point, then that is propagated to the caller.

There is actually a bit more to it. Mainly:

- The instance check is somewhat less dynamic than `isinstance` would do. For example, if `__new__` returns an object that is only an instance of the class you called because its type was registered as a virtual subclass of it, `__init__` is not called (nor would you probably want it to be).

- If `__init__` is called and returns any object other than `None`, then `TypeError` is raised, since that is always a bug. (`__init__` is conceptually "void." It should implicitly return `None`.)

Also, remember that a metaclass can customize this behavior by overriding `__call__`. This is fairly uncommon, but an example of it is `enum.EnumMeta`, the metaclass of `enum.Enum`.

## `__new__` returning existing instances: Is it okay?

An attempt to construct an object, whether by the usual means of calling the class or by other means, may return an existing object so long as doing so is consistent with all guarantees and reasonable expectations.

**When a type is mutable**&mdash;in the sense that its instances hold state that is permitted to change and that, if it changes, can cause formerly equal instances to become unequal&mdash;you **must not** have `__new__` return an existing instance on construction. For example, calling `list` is guaranteed to always give a new list&mdash;as is a list display like `[1, 2, 3]`. In contrast, `tuple` may hand us back an instance that already existed, and evaluating `(1, 2, 3)` twice may give us the same `tuple` object or different ones (that are equal).

**When a type's instances otherwise hold mutable state that is assumed separate**, then you likewise **must not** have `__new__` return an existing instance. For example, iterators almost always use reference-based equality comparison, and as such, their *value* never changes. But the whole point of an iterator is that it represents a particular step in a specific ongoing process of iteration: it keeps track of *where*, in the (eager or lazy) collection being iterated through, it currently is. So the basic documented guarantees about how to use an iterator would break if code that tries to construct an new iterator instead receives an existing iterator instance.

**Otherwise, it is usually acceptable** to have `__new__` return an existing instance instead of constructing a new one. Usually, this would only be an optimization, and it is not usually worthwhile. But sometimes there is a benefit, and sometimes you even want to allow the caller to rely on extra instances not being created.

## `__new__` returning existing instances: Cases

The above addresses when it is *permitted* to have `__new__` return an existing instance, in the sense of when it is *not guaranteed to be a severe design bug to do so.* But that does not address the details of when, in practice, we tend to consider it. Note that we don't usually bother with this, and in most situations it would either be unnecessary overkill or (due to the considerations discussed above) outright wrong.

There are a few major situations where we may decide to have `__new__` return objects that it did not actually cause to be newly constructed, but instead that existed previously.

#### 1. From the arguments to `__new__`

The exact object that should be returned, instead of constructing a new one, is immediately known from, or can be efficiently determined just by looking at, *the argument(s) to the call*&mdash;without having to check a cache of instances or other global state.

There are typically no disadvantages or complexities associated with concurrency, in these cases.

`decorators.linear_combinable` is an example of this technique: when the argument passed to `linear_combinable` is itself an instance of `linear_combinable`, there is no need to actually construct a new instance.

#### 2. A fixed number of stored instances, permanent once created

We intend that only a small number of instances are ever created and none need ever be destroyed. We create them all immediately (eager), or we create them on demand (lazy), and we keep track of them in state within or accessible from the class, usually in class attributes but occasionally in some (other) data structure.

When allowing instances to be *lazily* constructed&mdash;rather than constructing them at the time (or immediately after) the class is created&mdash;we must consider concurrency implications.

Singletons (see shortly below) and enumerations are examples of this technique.

#### 3. Unlimited, with weak caching

Many instances may be created, and are also allowed to be destroyed (that is, garbage collected). But we decide that multiple instances that have the same value (that is, that would be equal) should not exist at the same time&mdash;and that, whenever that *would* happen, the appropriate existing instance should be returned instead of creating a new one.

This technique requires that we keep track of our instances, *but with a light touch*: without preventing them from being garbage collected.

Concurrency implications must always be considered when applying this technique. If multiple threads may use the type (which in many applications, and especially in reusable libraries, is likely), then special efforts must be undertaken to ensure thread-safety. If performance is the goal of using this technique, then considerable effort may be needed to balance the goals of efficiency and thread-safety. Sometimes this will end up being an unsuitable technique, for that reason.

The most impressively useful application of this technique is [hash consing](https://en.wikipedia.org/wiki/Hash_consing), which can substantially improve space usage and running times of some recursive data structures such as trees, by ensuring that shared substructures (e.g., subtrees) are *always* reused, typically even across conceptually unrelated occurrences of the data structure. Hash consing may be viewed as a (particularly powerful) form of memoization. Of course, this is still only acceptable if the data structures are immutable: if mutable trees shared a subtree, then modifying one tree would inadvertently modify the other.

**FIXME**: Implement `greet.UniqueGreeter` and `sll.HashNode` (in that order). After doing so, remove this FIXME but briefly mention them in appropriate places in the preceding text.

## Singletons

With `__new__`, we can make a class that is limited to (at most) one instance.

In [129]:
class Singleton:
    """Lazy implementation of the singleton pattern. Not thread-safe."""
    
    _instance = None
    
    def __new__(cls): 
        if cls._instance is None: 
            cls._instance = super().__new__(cls)
        return cls._instance

In [130]:
a = Singleton()

In [131]:
b = Singleton()

In [132]:
a is b

True

In [133]:
c = Singleton()

In [134]:
c is a

True

In [135]:
class EagerSingleton:
    """Eager implementation of the singleton pattern. Thread safe."""
    
    _instance = None
    
    def __new__(cls): 
        if cls._instance is None: 
            cls._instance = super().__new__(cls)
        return cls._instance
    
EagerSingleton()  # __new__ assigns to class attribute

In [136]:
a = EagerSingleton()
b = EagerSingleton()
a is b 

True

In [137]:
class EagerSingletonExplicit:
    """Eager implementation of the singleton pattern. Thread safe."""
    
    _instance = None
    
    def __new__(cls): 
        if cls._instance is None: 
            cls._instance = super(EagerSingletonExplicit, cls).__new__(cls)
        return cls._instance
    
EagerSingletonExplicit()  # __new__ assigns to class attribute

In [138]:
a = EagerSingletonExplicit()
b = EagerSingletonExplicit()
a is b 

True

In [139]:
class EagerSingletonAlt: 
    """Alternate eager implementation of the singleton pattern. Thread safe."""
    
    def __new__(cls): 
        return cls._instance

EagerSingletonAlt._instance = super(EagerSingletonAlt, EagerSingletonAlt).__new__(EagerSingletonAlt)

In [140]:
a = EagerSingletonAlt()
b = EagerSingletonAlt()
a is b 

True

In [141]:
class EagerSingletonAltNoSuper: 
    """Alternate eager implementation of the singleton pattern. Thread safe."""
    
    def __new__(cls): 
        return cls._instance

# We know there is exactly one direct base class, object.
EagerSingletonAltNoSuper._instance = object.__new__(EagerSingletonAltNoSuper)

In [142]:
a = EagerSingletonAltNoSuper()
b = EagerSingletonAltNoSuper()
a is b 

True

## Singleton pattern with locking

In [143]:
from threading import Lock

In [144]:
class SingletonSafe:
    """Lazy thread-safe implementation of the singleton pattern."""
    
    _lock = Lock()
    _instance = None
    
    def __new__(cls): 
        with cls._lock: 
            if cls._instance is None: 
                cls._instance = super().__new__(cls)
            return cls._instance

In [145]:
a = SingletonSafe()
b = SingletonSafe()
a is b 

True

## The double-checked locking (anti?)pattern

In [146]:
class SingletonDC:
    """Lazy thread-safe(?) implementation of the singleton pattern."""
    
    _lock = Lock()
    _instance = None
    
    def __new__(cls): 
        if cls._instance is None:
            with cls._lock: 
                if cls._instance is None: 
                    cls._instance = super().__new__(cls)
        return cls._instance

In [147]:
a = SingletonDC()
b = SingletonDC()
a is b 

True

This need not be thread safe on all implementations:

1. If binding an attribute to an object reference is not an *atomic* operation, `cls._instance` may be accessed while it is being written to, and thus not contain a valid reference to `None` or a new instance. 

2. Unless memory order is guaranteed between threads, the attribute `cls._instance` may contain a correct reference in memory even though the data for the instance hasn't been written yet. Then a thread may attempt to use the `SingletonDC` instance before its data are available to code on that thread.

Related: https://www.cs.umd.edu/~pugh/java/memoryModel/DoubleCheckedLocking.html

In CPython, however, the double-checked locking pattern is guaranteed thread-safe due to the semantics of the GIL (global interpreter lock).

## Singletons and threads

See `datarace.py` for a singleton example with multithreading.

## Related meanings of "singleton"

1. We are using "singleton" to mean a class that has at most one instance (and not by coincidence, but because the code is written in such a way as to achieve this). For example, `NoneType` (which we can get by evaluating `type(None)`) is a singleton (i.e., a singleton class).

2. People sometimes use "singleton" to refer to the instance itself. For example, `None` is a singleton in this sense (it is a singleton object, i.e., a singleton instance, an instance of a singleton class).

3. People sometimes also use "singleton" to refer to instances of a class that is strictly limited to a fixed number of values beforehand with exactly one instance per value. This usage is perhaps unfortunate. For example, `True` and `False` are often said to be "singletons," even though they are instances of a class that is guaranteed to have exactly *two* instances.

## Making types limited to a few instances of known values

This is what enumerations (`enum.Enum`) are for.

In this project, `enumerations.ipynb` and `enumerations.py` have general information and examples of what enumerations in Python can do. For code specific to using enumerations for something that is conceptually not just an enumerator, see `greet.EnumGreeter`.

## Customizing attribute access

This is done with `__getattribute__`, `__getattr__`, `__setattr__`, and `__delattr__`.

See `greet.EnumGreeter`, `decorators.linear_combinable`, and `test_context._AttributeSpy`.